In [75]:
import locale
import pandas as pd
import geopandas as gpd
# Set locale to Turkish
locale.setlocale(locale.LC_COLLATE, 'tr_TR.UTF-8')

'tr_TR.UTF-8'

# Provinces

In [76]:
gdf_borders = gpd.read_file("data/raw/gdf_borders_ibbs3.geojson")
gdf_borders.sort_values(by="province",  inplace=True,key = lambda x:[locale.strxfrm(y) for y in x])
gdf_borders.reset_index(inplace=True,drop=True)
gdf_borders["id"] = gdf_borders.index
gdf_borders.head()

,province,geometry,id
0,Adana,"MULTIPOLYGON (((35.27903 36.64069, 35.27903 36...",0
1,Adıyaman,"POLYGON ((38.65534 38.09172, 38.66739 38.0916,...",1
2,Afyonkarahisar,"POLYGON ((31.36277 39.16468, 31.36626 39.15131...",2
3,Ağrı,"POLYGON ((43.39566 39.9258, 43.39598 39.91575,...",3
4,Aksaray,"POLYGON ((34.20521 38.84115, 34.21472 38.82775...",4


In [77]:
df_codes = pd.read_excel("data/raw/region_codes.xlsx",index_col=-1)
df_codes.head(2)

,ibbs1 code,region,ibbs2 code,sub-region,ibbs3 code
province,,,,,
İstanbul,TR1,İstanbul,TR10,İstanbul alt bölgesi,TR100
Tekirdağ,TR2,Batı Marmara,TR21,Tekirdağ alt bölgesi,TR211


In [78]:
df_codes = df_codes.ffill()
df_codes.head(2)

,ibbs1 code,region,ibbs2 code,sub-region,ibbs3 code
province,,,,,
İstanbul,TR1,İstanbul,TR10,İstanbul alt bölgesi,TR100
Tekirdağ,TR2,Batı Marmara,TR21,Tekirdağ alt bölgesi,TR211


In [79]:
df_codes[["sub-region"]] =df_codes[["sub-region"]].astype(str)
df_codes["sub-region"] = df_codes["sub-region"].map(lambda x: x.replace("alt bölgesi","\nsub-region"))
df_codes.head(2)

,ibbs1 code,region,ibbs2 code,sub-region,ibbs3 code
province,,,,,
İstanbul,TR1,İstanbul,TR10,İstanbul \nsub-region,TR100
Tekirdağ,TR2,Batı Marmara,TR21,Tekirdağ \nsub-region,TR211


In [80]:
region_dict = {"İstanbul":"Istanbul","Batı Marmara":"Western Marmara","Ege":"Aegean","Doğu Marmara":"Eastern Marmara","Batı Anadolu":"Western Anatolia",
               "Akdeniz":"Mediterranean","Orta Anadolu":"Central Anatolia","Batı Karadeniz":"Western Black Sea","Doğu Karadeniz":"Eastern Black Sea",
          "Kuzeydoğu Anadolu":"Northeastern Anatolia","Ortadoğu Anadolu":"Middle East Anatolia","Güneydoğu Anadolu":"Southeastern Anatolia"}

In [81]:
df_codes["region"] = df_codes["region"].map(region_dict)
df_codes.head(2)

,ibbs1 code,region,ibbs2 code,sub-region,ibbs3 code
province,,,,,
İstanbul,TR1,Istanbul,TR10,İstanbul \nsub-region,TR100
Tekirdağ,TR2,Western Marmara,TR21,Tekirdağ \nsub-region,TR211


In [82]:
gdf_borders = gdf_borders.merge( df_codes, on="province", how="left" )
gdf_borders.head(2)

,province,geometry,id,ibbs1 code,region,ibbs2 code,sub-region,ibbs3 code
0,Adana,"MULTIPOLYGON (((35.27903 36.64069, 35.27903 36...",0,TR6,Mediterranean,TR62,Adana \nsub-region,TR621
1,Adıyaman,"POLYGON ((38.65534 38.09172, 38.66739 38.0916,...",1,TRC,Southeastern Anatolia,TRC1,Gaziantep \nsub-region,TRC12


In [83]:
gdf_borders.to_file("data/preprocessed/gdf_borders_ibbs3.geojson")
df_codes.to_csv("data/preprocessed/region_codes.csv")

# Districts

In [84]:
gdf_borders = gpd.read_file("data/raw/gdf_borders_district.geojson")
gdf_borders.head(2)

,district,province,geometry
0,Elbeyli,Kilis,"POLYGON ((37.3596 36.7307, 37.3631 36.7228, 37..."
1,Dodurga,Çorum,"POLYGON ((34.63663 40.89572, 34.61219 40.88676..."


In [85]:
gdf_borders.reset_index(inplace=True,drop=True)
gdf_borders["id"] = gdf_borders.index
gdf_borders.head(2)

,district,province,geometry,id
0,Elbeyli,Kilis,"POLYGON ((37.3596 36.7307, 37.3631 36.7228, 37...",0
1,Dodurga,Çorum,"POLYGON ((34.63663 40.89572, 34.61219 40.88676...",1


In [86]:
gdf_borders.to_file("data/preprocessed/gdf_borders_district.geojson")

IGNORE BELOW (TEST TO OBSERVE)

In [15]:
# Use the dissolve method to merge provinces into regions
gdf_borders = gdf_borders.dissolve(by='sub-region')
gdf_borders.head()

,geometry,province,ibbs1 code,region,ibbs2 code,ibbs3 code
sub-region,,,,,,
Adana \nsub-region,"MULTIPOLYGON (((35.2793 36.64014, 35.27958 36....",Adana,TR6,Akdeniz,TR62,TR621
Ankara \nsub-region,"POLYGON ((33.18796 40.41365, 33.1941 40.40469,...",Ankara,TR5,Batı Anadolu,TR51,TR511
Antalya \nsub-region,"MULTIPOLYGON (((29.64097 36.12069, 29.64097 36...",Antalya,TR6,Akdeniz,TR61,TR611
Aydın \nsub-region,"MULTIPOLYGON (((27.33153 37.3307, 27.3318 37.3...",Aydın,TR3,Ege,TR32,TR321
Ağrı \nsub-region,"POLYGON ((43.39566 39.9258, 43.39598 39.91575,...",Ağrı,TRA,Kuzeydoğu Anadolu,TRA2,TRA21


In [39]:
gdf_borders.shape

(26, 6)

In [15]:
df_data = pd.read_csv("data/preprocessed/population/population-age-sex-2007-2023-ibbs3.csv", index_col=[0, 1], header=[0, 1])
df_data.head()

sex                   male                                                    \
age_group              0-4   10-14  15-19  20-24  25-29  30-34  35-39  40-44   
year province                                                                  
2007 Adana           87875  100851  96648  78022  90764  80670  70980  68068   
     Adıyaman        31114   35241  32872  24563  25786  20867  17573  15887   
     Afyonkarahisar  28518   31309  30986  32886  29935  25038  23980  22329   
     Aksaray         16675   18777  17846  14831  16510  14419  12434  11318   
     Amasya          10933   13796  13929  18042  13775  11303  10811  11107   

sex                                ... female                              \
age_group            45-49  50-54  ...  50-54  55-59    5-9  60-64  65-69   
year province                      ...                                      
2007 Adana           58558  52112  ...  51402  39610  93870  27931  22227   
     Adıyaman        11527  10253  ...  10555   9782  32072   7440   6242   
     Afyonkarahisar  21158  18181  ...  18780  15373  30085  12226  11654   
     Aksaray          8711   7629  ...   8186   7656  18367   5753   4778   
     Amasya          10029   9128  ...   9265   8309  12400   7230   6423   

sex                                                  
age_group            70-74  75-79 80-84 85-89   90+  
year province                                        
2007 Adana           16919  13775  7869  2810  1761  
     Adıyaman         4572   3973  2497  1103   656  
     Afyonkarahisar   9847   8158  3812   983   485  
     Aksaray          3707   2632  1527   567   314  
     Amasya           5633   4356  2146   594   284  

[5 rows x 38 columns]

In [33]:
res=gdf_borders.merge(tt, left_on="province", right_on="province")
res.head()

,province,geometry,ibbs1 code,region,ibbs2 code,sub-region,ibbs3 code,0
0,Çanakkale,"MULTIPOLYGON (((26.05347 39.84736, 26.05347 39...",TR2,Batı Marmara,TR22,Balıkesir alt bölgesi,TR222,12601
1,Çankırı,"POLYGON ((33.78607 40.96249, 33.78677 40.95684...",TR8,Batı Karadeniz,TR82,Kastamonu alt bölgesi,TR822,6038
2,Çorum,"POLYGON ((34.81906 41.20172, 34.84486 41.19581...",TR8,Batı Karadeniz,TR83,Samsun alt bölgesi,TR833,20091
3,Adana,"MULTIPOLYGON (((35.27903 36.64069, 35.27903 36...",TR6,Akdeniz,TR62,Adana alt bölgesi,TR621,87875
4,Adıyaman,"POLYGON ((38.65534 38.09172, 38.66739 38.09160...",TRC,Güneydoğu Anadolu,TRC1,Gaziantep alt bölgesi,TRC12,31114


In [22]:
tt=df_data.loc[2007,[("male","0-4")]].droplevel(1, axis=1).sum(axis=1).reset_index()

In [32]:
tt.head()

,province,0
0,Adana,87875
1,Adıyaman,31114
2,Afyonkarahisar,28518
3,Aksaray,16675
4,Amasya,10933


In [29]:
f=res.dissolve(by='sub-region')

In [30]:
f

,geometry,province,ibbs1 code,region,ibbs2 code,ibbs3 code,0
sub-region,,,,,,,
Adana alt bölgesi,"MULTIPOLYGON (((33.53125 36.12847, 33.53125 36...",Adana,TR6,Akdeniz,TR62,TR621,87875
Ankara alt bölgesi,"POLYGON ((33.18796 40.41365, 33.19410 40.40469...",Ankara,TR5,Batı Anadolu,TR51,TR511,161281
Antalya alt bölgesi,"MULTIPOLYGON (((29.64097 36.12069, 29.64097 36...",Antalya,TR6,Akdeniz,TR61,TR611,71077
Aydın alt bölgesi,"MULTIPOLYGON (((27.40681 36.74208, 27.40681 36...",Aydın,TR3,Ege,TR32,TR321,31432
Ağrı alt bölgesi,"POLYGON ((44.48705 39.68656, 44.48495 39.67772...",Ağrı,TRA,Kuzeydoğu Anadolu,TRA2,TRA21,34639
Balıkesir alt bölgesi,"MULTIPOLYGON (((26.60014 39.30208, 26.60042 39...",Çanakkale,TR2,Batı Marmara,TR22,TR222,12601
Bursa alt bölgesi,"MULTIPOLYGON (((31.01101 40.01844, 31.02343 40...",Bilecik,TR4,Doğu Marmara,TR41,TR413,6715
Erzurum alt bölgesi,"POLYGON ((40.60795 39.53184, 40.58783 39.53662...",Bayburt,TRA,Kuzeydoğu Anadolu,TRA1,TRA13,3252
Gaziantep alt bölgesi,"POLYGON ((37.50204 36.64929, 37.48498 36.64109...",Adıyaman,TRC,Güneydoğu Anadolu,TRC1,TRC12,31114


In [31]:
f.shape

(26, 7)